In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/anon-Jove/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.DotBashers import *
from jove.Def_md2mc  import *
from jove.Def_DFA    import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
ev0end1 = md2mc('''
DFA
I : 0 -> A
A : 0 | 1 -> I
I : 1 -> F
F : 0 | 1 -> I
''')

In [ ]:
doev0end1 = dotObj_dfa(ev0end1)

In [ ]:
ev0end1

In [ ]:
doev0end1.source

In [ ]:
is_partially_consistent_dfa(ev0end1)

In [ ]:
tev0end1 = totalize_dfa(ev0end1)

In [ ]:
dotObj_dfa_w_bh(tev0end1)

In [ ]:
dotObj_dfa_w_bh(tev0end1, FuseEdges=True)

In [ ]:
ev0end1

In [ ]:
ev0 = md2mc('''
DFA
IF : 0 -> A
A  : 0 -> IF
''')

In [ ]:
ev0

In [ ]:
dev0 = dotObj_dfa(ev0)

In [ ]:
dev0

In [ ]:
dev0.source

In [ ]:
dev0

In [ ]:
ev0_bh =  addtosigma_dfa(ev0, set({'1'}))

In [ ]:
ev0_bh

In [ ]:
ev0_bh_totalize = totalize_dfa(ev0_bh)

In [ ]:
ev0_bh

In [ ]:
do_ev0_tot = dotObj_dfa_w_bh(ev0_bh_totalize)

In [ ]:
do_ev0_tot.source

In [ ]:
do_ev0_tot

In [ ]:
dotObj_dfa_w_bh(ev0_bh_totalize, FuseEdges=True)

<span style="color:blue"> **Here is how we will represent a DFA in Python (taking Figure 3.4's example from the book). You can clearly see how the traits of the DFA are encoded. We prefer a Python dictionary, as it supports a number of convenient operations, and also one can add additional fields easily. ** </span>

In [ ]:
DFA_fig34 = { 'Q': {'A', 'IF', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('IF', '0'): 'A',
                         ('IF', '1'): 'IF',
                         ('A', '0'): 'B',
                         ('A', '1'): 'A',
                         ('B', '0'): 'IF',
                         ('B', '1'): 'B' },
              'q0': 'IF', 
              'F': {'IF'}   
            }

<span style="color:blue"> **We can now write routines to print DFA using dot. The main routines are listed below.** </span>

* dot_dfa_w_bh : lists all states of a DFA including black-hole states
* dot_dfa      : lists all isNotBH states (see below for a defn), i.e. suppress black-holes
     - Usually there are too many transitions to them and that clutters the view
     

======

In [ ]:
# Some tests pertaining to totalize_dfa, is_consistent_dfa, etc

DFA_fig34 = { 'Q': {'A', 'IF', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('IF', '0'): 'A',
                         ('IF', '1'): 'IF',
                         ('A', '0'): 'B',
                         ('A', '1'): 'A',
                         ('B', '0'): 'IF',
                         ('B', '1'): 'B' },
              'q0': 'IF', 
              'F': {'IF'}   
            }

def tests_dfa_consist():
    """Some tests wrt DFA routines.
    """
    DFA_fig34_Q     = DFA_fig34["Q"]
    DFA_fig34_Sigma = DFA_fig34["Sigma"]
    randQ           = random.choice(list(DFA_fig34_Q))
    randSym         = random.choice(list(DFA_fig34_Sigma))

    DFA_fig34_deepcopy  = copy.deepcopy(DFA_fig34)

    print('is_consistent_dfa(DFA_fig34) =', 
           is_consistent_dfa(DFA_fig34) )
    print('Removing mapping for ' + 
          "(" + randQ + "," + randSym + ")" + 
          "from DFA_fig34_deepcopy")
    DFA_fig34_deepcopy["Delta"].pop((randQ,randSym))
    print('is_consistent_dfa(DFA_fig34_deepcopy) =', 
         is_consistent_dfa(DFA_fig34_deepcopy) )

    totalized = totalize_dfa(DFA_fig34_deepcopy)
    print ( 'is_consistent_dfa(totalized) =', 
          is_consistent_dfa(totalized) )

    assert(totalized == totalize_dfa(totalized)) # Must pass  

In [ ]:
dfaBESame = md2mc('''
DFA !! Begins and ends with same; epsilon allowed
IF  : 0 -> F0
IF  : 1 -> F1
!!
F0  : 0 -> F0
F0  : 1 -> S01
S01 : 1 -> S01
S01 : 0 -> F0
!!
F1  : 1 -> F1
F1  : 0 -> S10
S10 : 0 -> S10
S10 : 1 -> F1
''')
DOdfaBESame = dotObj_dfa(dfaBESame)
DOdfaBESame

In [ ]:
DOdfaBESame.source

### Let us now administer some tests to print dot-strings generated.

We will demonstrate two ways to print automata: 

1. First generate a dot string via dot_dfa or dot_dfa_w_bh
   (calling the result "dot_string") 
   1. Then use the srcObj = Source(dot_string) call
   2. Thereafter we can display the srcObj object directly into the browser
   3. Or, one can also later convert the dot_string to svg or PDF
2. OR, one can directly generate a dot object via the dotObj_dfa or dotObj_dfa_w_bh call
   (calling the result "dot_object")
   1. Then directly display the dot_object
   2. There are conversions available for dot_object to other formats too

In [ ]:
DFA_fig34 = { 'Q': {'A', 'IF', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('IF', '0'): 'A',
                         ('IF', '1'): 'IF',
                         ('A', '0'): 'B',
                         ('A', '1'): 'A',
                         ('B', '0'): 'IF',
                         ('B', '1'): 'B' },
              'q0': 'IF', 
              'F': {'IF'}   
            }

def dfa_dot_tests():
    """Some dot-routine related tests.
    """
    dot_string = dot_dfa(DFA_fig34)
    dot_object1 = Source(dot_string)
    return dot_object1.source

Let us test functions step_dfa, run_dfa, and accepts_dfa

In [ ]:
# Some tests of step, run, etc.

DFA_fig34 = { 'Q': {'A', 'IF', 'B'},
              'Sigma': {'0', '1'},
              'Delta': { ('IF', '0'): 'A',
                         ('IF', '1'): 'IF',
                         ('A', '0'): 'B',
                         ('A', '1'): 'A',
                         ('B', '0'): 'IF',
                         ('B', '1'): 'B' },
              'q0': 'IF', 
              'F': {'IF'}   
            }

def step_run_accepts_tests():
    print("step_dfa(DFA_fig34, 'IF', '1') = ", 
          step_dfa(DFA_fig34, 'IF', '1'))
    print("step_dfa(DFA_fig34, 'A', '0') = ", 
          step_dfa(DFA_fig34, 'A', '0'))

    print("run_dfa(DFA_fig34, '101001') = ", 
          run_dfa(DFA_fig34, '101001'))
    print("run_dfa(DFA_fig34, '101000') = ", 
          run_dfa(DFA_fig34, '101000'))

    print("accepts_dfa(DFA_fig34, '101001') = ", 
          accepts_dfa(DFA_fig34, '101001')) 
    print("accepts_dfa(DFA_fig34, '101000') = ", 
          accepts_dfa(DFA_fig34, '101000')) 

In [ ]:
dotObj_dfa(DFA_fig34, "DFA_fig34")

In [ ]:
# Run a complementation test
DFA_fig34_comp = comp_dfa(DFA_fig34)
dotObj_dfa(DFA_fig34_comp, "DFA_fig34_comp")
dotObj_dfa(DFA_fig34)
dotObj_dfa(DFA_fig34_comp, "DFA_fig34_comp")

In [ ]:
dotObj_dfa(DFA_fig34_comp)

In [ ]:
# One more test
du     = union_dfa(DFA_fig34, DFA_fig34_comp)
dotObj_dfa(du, "orig union")
pdu    = pruneUnreach(du)
pdu
pduObj = dotObj_dfa(pdu, "union of 34 and comp")
pduObj



In [ ]:
D34 = {   'Q': {'A', 'IF', 'B'},
          'Sigma': {'0', '1'},
          'Delta': { ('IF', '0'): 'A',
                     ('IF', '1'): 'IF',
                     ('A', '0'): 'B',
                     ('A', '1'): 'A',
                     ('B', '0'): 'IF',
                     ('B', '1'): 'B' },
          'q0': 'IF', 
          'F': {'IF'}   
        }

D34bl = { 'Q': {'A', 'IF', 'B', 'A1', 'B1'},
          'Sigma': {'0', '1'},
          'Delta': { ('IF', '0'): 'A',
                     ('IF', '1'): 'IF',
                     ('A', '0'): 'B1',
                     ('A', '1'): 'A1',
                     ('A1', '0'): 'B',
                     ('A1', '1'): 'A',
                     ('B1', '0'): 'IF',
                     ('B1', '1'): 'B',
                     ('B','0') : 'IF',
                     ('B', '1'): 'B1' },
          'q0': 'IF', 
          'F': {'IF'}   
        }


d34 = dotObj_dfa(D34, "D34")
d34 # Display it!

In [ ]:
langeq_dfa(D34,D34bl,False)

In [ ]:
iso_dfa(D34,D34bl)

In [ ]:
DFA_fig34
d34 = DFA_fig34
d34

In [ ]:
d34c = DFA_fig34_comp
d34c

In [ ]:
iso_dfa(d34,d34)

In [ ]:
iso_dfa(d34,d34c)

In [ ]:
d34v1 = {'Delta': {('A', '0'): 'B',
  ('A', '1'): 'B',
  ('B', '0'): 'IF',
  ('B', '1'): 'B',
  ('IF', '0'): 'A',
  ('IF', '1'): 'IF'},
 'F': {'IF'},
 'Q': {'A', 'B', 'IF'},
 'Sigma': {'0', '1'},
 'q0': 'IF'}

In [ ]:
dotObj_dfa(d34v1)

In [ ]:
d34v2 = {'Delta': {('A', '0'): 'B',
  ('A', '1'): 'B',
  ('B', '0'): 'IF',
  ('B', '1'): 'B',
  ('IF', '0'): 'A',
  ('IF', '1'): 'IF'},
 'F': {'IF', 'B'},
 'Q': {'A', 'B', 'IF'},
 'Sigma': {'0', '1'},
 'q0': 'IF'}

In [ ]:
iso_dfa(d34,d34v1)

In [ ]:
iso_dfa(d34,d34v2)

In [ ]:
iso_dfa(d34v1,d34v2)

In [ ]:
div1 = pruneUnreach(intersect_dfa(d34v1,d34v2))
dotObj_dfa(div1)

In [ ]:
div2 = pruneUnreach(union_dfa(d34v1,d34v2))
dotObj_dfa(div2)

In [ ]:
iso_dfa(div1,div2)

In [ ]:
langeq_dfa(div1,div2,True)

In [ ]:
d34bl = dotObj_dfa(D34bl, "D34bl")
d34bl # Display it!

In [ ]:
d34bl = dotObj_dfa(D34bl, FuseEdges=True, dfaName="D34bl")

In [ ]:
d34bl

In [ ]:
iso_dfa(D34,D34bl)

In [ ]:
langeq_dfa(D34,D34bl)

#### 

In [ ]:
du

In [ ]:
dotObj_dfa(pruneUnreach(D34bl), "D34bl")

In [ ]:
### DFA minimization (another example)

In [ ]:
Bloat1 = {'Q': {'S1', 'S3', 'S2', 'S5', 'S4', 'S6' },
          'Sigma': {'b', 'a'},
          'Delta': { ('S1','b') : 'S3',
                     ('S1','a') : 'S2',
                     ('S3','a') : 'S5',
                     ('S2','a') : 'S4',
                     ('S3','b') : 'S4',
                     ('S2','b') : 'S5',
                     ('S5','b') : 'S6',
                     ('S5','a') : 'S6',
                     ('S4','b') : 'S6',
                     ('S4','a') : 'S6',
                     ('S6','b') : 'S6',
                     ('S6','a') : 'S6' },
          'q0': 'S1', 
          'F': {'S2','S3','S6'}   
        }


Bloat1O = dotObj_dfa(Bloat1, dfaName="Bloat1")
Bloat1O # Display it!

In [ ]:
dotObj_dfa(Bloat1, FuseEdges=True, dfaName="Bloat1")

In [ ]:
bloated_dfa = md2mc('''
DFA
IS1 : a -> FS2
IS1 : b -> FS3
FS2 : a -> S4
FS2 : b -> S5
FS3 : a -> S5
FS3 : b -> S4
S4  : a | b -> FS6
S5  : a | b -> FS6
FS6 : a | b -> FS6
''')
dotObj_dfa(bloated_dfa)

In [ ]:
dotObj_dfa(bloated_dfa).source


Now, here is how the computation proceeds for this example:
-------------------------------------------------------- 
 
 <br>
 
<font size="3"> 


```
 
Frame-0                  Frame-1                   Frame-2                    
                                                                                                     
S2  -1                   S2   0                    S2   0                     

S3  -1  -1               S3   0  -1                S3   0  -1                 

S4  -1  -1  -1           S4  -1   0   0            S4   2   0   0             

S5  -1  -1  -1  -1       S5  -1   0   0  -1        S5   2   0   0  -1         

S6  -1  -1  -1  -1  -1   S6   0  -1  -1   0   0    S6   0   1   1   0   0     

    S1  S2  S3  S4  S5       S1  S2  S3  S4  S5        S1  S2  S3  S4  S5        

Initial                  0-distinguishable         1-distinguishable                         
     
     
Frame-3                 Frame-4     
                        =
                        Frame-3

S2   0

S3   0  -1

S4   2   0   0

S5   2   0   0  -1

S6   0   1   1   0   0

    S1  S2  S3  S4  S5
    
2-distinguishable 
     
```
</font>

Here is the algorithm, going frame by frame.

- Initial Frame: 

     The initial frame is drawn to clash all _combinations_ of states taken two at a time.
     Since we have 6 states, we have $6\choose 2$ = $15$ entries. We put a -1 against each
     such pair to denote that they have not been found distinguishable yet.

- Frame *0-distinguishable*: We now put a 0 where a pair of states is 0-distinguishable. This means the states are distinguisable after consuming $\varepsilon$. This of course means that the states are themselves distinguishable. This is only possible if one is a final state and the other is not (in that case, one state, after consuming $\varepsilon$ accepts_dfa, and another state after consuming $\varepsilon$ does not accept.

  - So for instance, notice that (S3,S1) and (S4,S2) are 0-distinguishable, meaning that one is a final and the other is a non-final state.

- Frame *1-distinguishable*: We now put a 1 where a pair of states is 1-distinguishable. This means the states are distinguisable after consuming a string of length $1$ (a single symbol). This is only possible if one state transitions to a final state and the other transitions to a non-final state after consuming a member of $\Sigma$. 

  State pairs (S6,S2) and (S6,S3) are of this kind. While both S6 and S2 are final states (hence _0-indistinguishable_), after consuming an 'a' (or a 'b') they respectively go to a final/non-final state.
 This means that

  - after processing **the same symbol** one state -- let's say pre_p -- finds itself landing in a state p and another state  -- let's say pre_q -- finds itself landing in a state q such that (p,q) is 0-distinguishable.
  
  - When this happens, states pre-p and pre-q are **1-distinguishable**.

- Frame *2-distinguishable*: We now put a 2 where a pair of states is 2-distinguishable. This means the states are distinguisable after consuming a string of length $2$ (a string of length $2$). This is only possible if one state transitions to a state (say p) and the other transitions to state (say q) after consuming a member of $\Sigma$ such that (p,q) is **1-distinguishable**. State pairs (S5,S1) and (S4,S1) are 2-distinguishable because

  - after processing **the same symbol** one state -- let's say pre_p -- finds itself landing in a state p and another state  -- let's say pre_q -- finds itself landing in a state q such that (p,q) is 0-distinguishable.
  
  - When this happens, states pre-p and pre-q are **1-distinguishable**.
  
  - One example is this:
  
    - S5 and S1 are 2-distinguishable.
    
    - This is because after seeing an 'aa', S1 lands in a non-final state while S5 lands in a final state
    
    - Observe that "aa" = "a" + "a" . Thus, after eating the first "a", S1 lands in S2 while S5 lands in S6, and (S2,S6) have already been deemed 1-distinguishable.
    
    - Thus, when we mark (S5,S1) as 2-distinguishable, we are sending the matrix entry at (S5,S2) from 
      -1 to 2
 


  - Now, in search of 3-distinguishability, we catch hold of all pairs in the matrix and see if we can send another -1 entry to "3". This appears not to happen. 
  
     - Thus, if (S2,S3) is pushed via any sequence of symbols (any string) of any length, it
       always stays in the same type of state. Thus, after seeing 'ababba', S2 is in S6, while S3 
        is also in S6.


 - Thus, given no changes in the matrix, we stop.

In [ ]:
dotObj_dfa(min_dfa(Bloat1), FuseEdges=True, dfaName="shrunkBloat1")

In [ ]:
min_bloat = min_dfa(Bloat1)
dotObj_dfa(min_bloat).source

In [ ]:
prd34b1 = pruneUnreach(D34bl)

In [ ]:
dotObj_dfa(prd34b1, "prd34b1")

In [ ]:
dotObj_dfa(min_dfa(prd34b1), "prd34b1min")

In [ ]:
third1dfa=md2mc(src="File", fname="machines/dfafiles/thirdlastis1.dfa")

In [ ]:
third1dfa

In [ ]:
dotObj_dfa(third1dfa)

In [ ]:
ends0101 =\
"\
DFA\
 \
I     : 0 -> S0 \
I     : 1 -> I \
S0    : 0 -> S0 \
S0    : 1 -> S01 \
S01   : 0 -> S010 \
S01   : 1 -> I \
S010  : 0 -> S0 \
S010  : 1 -> F0101 \
F0101 : 0 -> S010 \
F0101 : 1 -> I \
"

In [ ]:
ends0101

In [ ]:
dfaends0101=md2mc(ends0101)

In [ ]:
dfaends0101

In [ ]:
dped1 = md2mc(src="File", fname="machines/dfafiles/pedagogical1.dfa")
#machines/dfafiles/pedagogical1.dfa

In [ ]:
dped1

In [ ]:
dotObj_dfa(dped1)

In [ ]:
dotObj_dfa(dped1, FuseEdges=True)

In [ ]:
dotObj_dfa(md2mc(ends0101))

In [ ]:
thirdlastis1=md2mc(src="File", fname="machines/dfafiles/thirdlastis1.dfa")
#machines/dfafiles/thirdlastis1.dfa

In [ ]:
thirdlastis1

In [ ]:
dotObj_dfa(thirdlastis1)

In [ ]:
dped1=md2mc(src="File", fname="machines/dfafiles/pedagogical2.dfa")
#machines/dfafiles/pedagogical2.dfa

In [ ]:
dotObj_dfa(dped1)

In [ ]:
secondLastIs1 = md2mc('''
!!------------------------------------------------------------
!! This DFA looks for patterns of the form ....1.
!! i.e., the second-last (counting from the end-point) is a 1
!!
!! DFAs find such patterns "very stressful to handle",
!! as they are kept guessing of the form  'are we there yet?'
!! 'are we seeing the second-last' ?
!! They must keep all the failure options at hand. Even after
!! a 'fleeting glimpse' of the second-last, more inputs can
!! come barreling-in to make that "lucky 1" a non-second-last.
!!
!! We take 7 states in the DFA solution.
!!------------------------------------------------------------

DFA
!!------------------------------------------------------------
!! State : in ->  tostate !! comment
!!------------------------------------------------------------

I   :  0 ->  S0  !! Enter at init state I
I   :  1 ->  S1  !! Record bit seen in state letter
                     !! i.e., S0 means "state after seeing a 0"
			 
S0  :  0 ->  S00 !! continue recording input seen
S0  :  1 ->  S01 !! in state-letter. This is a problem-specific
                 !! way of compressing the input seen so far.

S1  :  0 ->  F10 !! We now have a "second last" available!
S1  :  1 ->  F11 !! Both F10 and F10 are "F" (final)

S00 :  0 ->  S00 !! History of things seen is still 00
S00 :  1 ->  S01 !! Remember 01 in the state

S01 :  0 ->  F10 !! We again have a second-last of 1
S01 :  1 ->  F11 !! We are in F11 because of 11 being last seen

F10 :  0 ->  S00 !! The second-last 1 gets pushed-out
F10 :  1 ->  S01 !! The second-last 1 gets pushed-out here too

F11 :  0 ->  F10 !! Still we have a second-last 1
F11 :  1 ->  F11 !! Stay in F11, as last two seen are 11

!!------------------------------------------------------------
''')

In [ ]:
from math import floor, log, pow
def nthnumeric(N, Sigma={'a','b'}):
    """Assume Sigma is a 2-sized list/set of chars (default {'a','b'}). 
       Produce the Nth string in numeric order, where N >= 0.
       Idea : Given N, get b = floor(log_2(N+1)) - need that 
       many places; what to fill in the places is the binary 
       code for N - (2^b - 1) with 0 as Sigma[0] and 1 as Sigma[1].    
    """
    if (type(Sigma)==set):
       S = list(Sigma)
    else:
       assert(type(Sigma)==list
       ), "Expected to be given set/list for arg2 of nthnumeric."
       S = Sigma
    assert(len(Sigma)==2
          ),"Expected to be given a Sigma of length 2."
    if(N==0):
        return ''
    else:
        width = floor(log(N+1, 2))
        tofill = int(N - pow(2, width) + 1)
        relevant_binstr = bin(tofill)[2::] # strip the 0b 
                                           # in the leading string
        len_to_makeup = width - len(relevant_binstr)
        return (S[0]*len_to_makeup + 
                shomo(relevant_binstr,
                      lambda x: S[1] if x=='1' else S[0]))

In [ ]:
nthnumeric(20,['0','1'])

In [ ]:
run_dfa(secondLastIs1, '0101')

In [ ]:
accepts_dfa(secondLastIs1, '0101')

In [ ]:
tests = [ nthnumeric(i, ['0','1']) for i in range(12) ]
for t in tests:
    if accepts_dfa(secondLastIs1, t):
        print("This DFA accepts ", t)
    else:
        print("This DFA rejects ", t)

In [ ]:
help(run_dfa)

This is an extensive illustration of union, intersection and complementation, DFA minimization, isomorphism test, language equivalence test, and an application of DeMorgan's law

In [ ]:
dfaOdd1s = md2mc('''
DFA
I : 0 -> I
I : 1 -> F
F : 0 -> F 
F : 1 -> I
''')

In [ ]:
dotObj_dfa(dfaOdd1s)

In [ ]:
dotObj_dfa(dfaOdd1s).source

In [ ]:
ends0101 = md2mc('''
DFA 
I     : 0 -> S0  
I     : 1 -> I 
S0    : 0 -> S0 
S0    : 1 -> S01 
S01   : 0 -> S010 
S01   : 1 -> I 
S010  : 0 -> S0 
S010  : 1 -> F0101 
F0101 : 0 -> S010 
F0101 : 1 -> I 
''')

In [ ]:
dotObj_dfa(ends0101)

In [ ]:
dotObj_dfa(ends0101).source

In [ ]:
odd1sORends0101 = union_dfa(dfaOdd1s,ends0101)

In [ ]:
dotObj_dfa(odd1sORends0101)

In [ ]:
dotObj_dfa(odd1sORends0101)

In [ ]:
dotObj_dfa(odd1sORends0101).source

In [ ]:
Minodd1sORends0101 = min_dfa(odd1sORends0101)

In [ ]:
dotObj_dfa(Minodd1sORends0101)

In [ ]:
dotObj_dfa(Minodd1sORends0101).source

In [ ]:
iso_dfa(odd1sORends0101, Minodd1sORends0101)

In [ ]:
langeq_dfa(odd1sORends0101, Minodd1sORends0101)

In [ ]:
odd1sANDends0101 = intersect_dfa(dfaOdd1s,ends0101)

In [ ]:
dotObj_dfa(odd1sANDends0101)

In [ ]:
Minodd1sANDends0101 = min_dfa(odd1sANDends0101)

In [ ]:
dotObj_dfa(Minodd1sANDends0101)

In [ ]:
dotObj_dfa(Minodd1sANDends0101).source

In [ ]:
CdfaOdd1s = comp_dfa(dfaOdd1s)

In [ ]:
Cends0101 = comp_dfa(ends0101)

In [ ]:
C_CdfaOdd1sORCends0101 = comp_dfa(union_dfa(CdfaOdd1s, Cends0101))

In [ ]:
dotObj_dfa(C_CdfaOdd1sORCends0101)

In [ ]:
MinC_CdfaOdd1sORCends0101 = min_dfa(C_CdfaOdd1sORCends0101)

In [ ]:
dotObj_dfa(MinC_CdfaOdd1sORCends0101)

In [ ]:
iso_dfa(MinC_CdfaOdd1sORCends0101, Minodd1sANDends0101)

In [ ]:
blimp = md2mc('''
DFA 
I1 : a -> F2
I1 : b -> F3
F2 : a -> S8
F2 : b -> S5
F3 : a -> S7
F3 : b -> S4
S4 : a | b -> F6
S5 : a | b -> F6
F6 : a | b -> F6
S7 : a | b -> F6
S8 : a -> F6
S8 : b -> F9
F9 : a -> F9
F9 : b -> F6
''')

In [ ]:
dblimp = dotObj_dfa(blimp)

In [ ]:
dblimp

In [ ]:
dblimp = dotObj_dfa(blimp, FuseEdges=True)

In [ ]:
dblimp

In [ ]:
dblimp.source

In [ ]:
mblimp = min_dfa(blimp)

In [ ]:
dmblimp = dotObj_dfa(mblimp)

In [ ]:
dmblimp

This shows how DeMorgan's Law applies to DFAs. It also shows how, using the tools provided to us, we can continually check our work.

In [ ]:
testdfa = md2mc('''DFA
I : 0 -> I
I : 1 -> F
F : 0 -> I
''')

In [ ]:
testdfa

In [ ]:
tot_testdfa = totalize_dfa(testdfa)

In [ ]:
dotObj_dfa(tot_testdfa)

In [ ]:
dotObj_dfa_w_bh

In [ ]:
dotObj_dfa_w_bh(tot_testdfa)

In [ ]:
dotObj_dfa_w_bh(tot_testdfa, FuseEdges = True)